# Jores et al 2021 Extract-Transform-Load
**Authorship:**
Adam Klie, *08/11/2022*
***
**Description:**
Notebook to extract, transform, and load (ETL) data from the Jores et al (2021) dataset.
***

In [ ]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import eugene as eu
eu.settings.dataset_dir = "/cellar/users/aklie/data/eugene"

# Download and load in the dataset to a raw `SeqData` object

In [ ]:
# Load in the downloaded datasets from the manuscript Github repo
sdata_leaf_raw = eu.datasets.jores21(dataset="leaf")
sdata_proto_raw = eu.datasets.jores21(dataset="proto")

In [ ]:
# Download metadata, promoters to evolve and motifs in MEME format (TODO: add to jores21 datasets module function)
!wget https://static-content.springer.com/esm/art%3A10.1038%2Fs41477-021-00932-y/MediaObjects/41477_2021_932_MOESM3_ESM.xlsx -O /cellar/users/aklie/data/eugene/jores21/41477_2021_932_MOESM3_ESM.xlsx
!wget https://raw.githubusercontent.com/tobjores/Synthetic-Promoter-Designs-Enabled-by-a-Comprehensive-Analysis-of-Plant-Core-Promoters/main/analysis/validation_sequences/promoters_for_evolution.tsv -O /cellar/users/aklie/data/eugene/jores21/promoters_for_evolution.tsv
!wget https://raw.githubusercontent.com/tobjores/Synthetic-Promoter-Designs-Enabled-by-a-Comprehensive-Analysis-of-Plant-Core-Promoters/main/data/misc/CPEs.meme -O /cellar/users/aklie/data/eugene/jores21/CPEs.meme
!wget https://raw.githubusercontent.com/tobjores/Synthetic-Promoter-Designs-Enabled-by-a-Comprehensive-Analysis-of-Plant-Core-Promoters/main/data/misc/TF-clusters.meme -O /cellar/users/aklie/data/eugene/jores21/TF-clusters.meme

In [ ]:
# Concatenate the datasets into a combined SeqData object
sdata_combined_raw = eu.dl.concat([sdata_leaf_raw, sdata_proto_raw], keys=["leaf", "proto"])
sdata_combined_raw.make_names_unique()

In [ ]:
# Give a quick check of this dataset
sdata_combined_raw.seqs_annot.head()

In [ ]:
# Load some other sequence features
smetadata = pd.read_excel(
    os.path.join(eu.settings.dataset_dir, "jores21", "41477_2021_932_MOESM3_ESM.xlsx"),
    sheet_name=0, 
    skiprows=3, 
)
smetadata.drop(columns=["species", "gene"], inplace=True)
smetadata["sequence"].isin(sdata_combined_raw.seqs).sum()

In [ ]:
# Add the sequence annotations to the combined SeqData object
sdata_combined_raw["sequence"] = sdata_combined_raw.seqs
sdata_combined_raw.seqs_annot = sdata_combined_raw.seqs_annot.merge(smetadata, on="sequence", how="left")
sdata_combined_raw.seqs_annot.drop(columns=["sequence", "UTR"], inplace=True)

In [ ]:
# Save raw versions of these three
sdata_leaf_raw.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "leaf_raw.h5sd"))
sdata_proto_raw.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "proto_raw.h5sd"))
sdata_combined_raw.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "combined_raw.h5sd"))

# Transform the input data in the combined SeqData object

In [ ]:
# Add reverse complement sequences and one-hot encoded sequences (forward and reverse complement)
eu.pp.ohe_seqs_sdata(sdata_combined_raw)
eu.pp.reverse_complement_seqs_sdata(sdata_combined_raw)

In [ ]:
# Save the combined processed SeqData object
sdata_combined_raw.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "combined_processed.h5sd"))

In [ ]:
# Subset back to leaf and proto objects
sdata_leaf_processed = sdata_combined_raw[sdata_combined_raw["batch"] == "leaf"]
sdata_proto_processed = sdata_combined_raw[sdata_combined_raw["batch"] == "proto"]

In [ ]:
# Split each of the three datases into training and train sets
sdata_leaf_train = sdata_leaf_processed[sdata_leaf_processed["set"] == "train"]
sdata_proto_train = sdata_proto_processed[sdata_proto_processed["set"] == "train"]
sdata_combined_train = sdata_combined_raw[sdata_combined_raw["set"] == "train"]
sdata_leaf_test = sdata_leaf_processed[sdata_leaf_processed["set"] == "test"]
sdata_proto_test = sdata_proto_processed[sdata_proto_processed["set"] == "test"]
sdata_combined_test = sdata_combined_raw[sdata_combined_raw["set"] == "test"]

In [ ]:
# Add in a column signifying whether the sequence is in the training set or the validation set
eu.pp.train_test_split_sdata(sdata_leaf_train, train_key="train_val", split=0.9)
eu.pp.train_test_split_sdata(sdata_proto_train, train_key="train_val", split=0.9)

In [ ]:
# Save all the sets
sdata_leaf_processed.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "leaf_processed.h5sd"))
sdata_proto_processed.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "proto_processed.h5sd"))
sdata_leaf_train.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "leaf_processed_train.h5sd"))
sdata_proto_train.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "proto_processed_train.h5sd"))
sdata_combined_train.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "combined_processed_train.h5sd"))
sdata_leaf_test.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "leaf_processed_test.h5sd"))
sdata_proto_test.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "proto_processed_test.h5sd"))
sdata_combined_test.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "combined_processed_test.h5sd"))

---